## 1. Connect to Azure OpenAI LLM
Initialize and connect to the Azure OpenAI service using credentials from environment variables to enable chat completions.

In [16]:
import os
load_dotenv()

True

In [ ]:
#connect to llm
from openai import AzureOpenAI
from dotenv import load_dotenv


client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    azure_endpoint=os.getenv("OPENAI_BASE_URL"),
    api_version="2024-08-01-preview"
)

response = client.chat.completions.create(   
  model="gpt-4o",#  Replace with your actual deployment name from Azure Portal
  messages=[
    {"role": "user", "content": "This is a test."}
  ]
)

print(response.model_dump_json(indent=2))

## 2. Install Azure Search Documents Library
Install the required `azure-search-documents` package to interact with Azure AI Search service.
- pip install azure-search-documents

## 3. Connect to Azure AI Search Service
Initialize the SearchClient with endpoint, index name, and API key from environment variables to enable queries against the Azure Search index.

In [22]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

service_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
index_name = os.environ["AZURE_SEARCH_INDEX_NAME"]
key = os.environ["AZURE_SEARCH_API_KEY"]

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

## 4. Search and Display Results
Execute a search query against the Azure Search index and display all results with their fields and values.

In [ ]:
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType

# Semantic search with reranking
results = search_client.search(
    search_text="ROBERT AUDI",
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="default",  # or your configured semantic config name
    top=5  # number of results to return
)

print("Semantic Search Results...")
result_list = list(results)
print(f"Number of results: {len(result_list)}")

if result_list:
    print("\nFirst result:")
    print(result_list[0])
    
    for i, result in enumerate(result_list):
        print(f"\n--- Result {i+1} ---")
        for key, value in result.items():
            print(f"{key}: {value}")
else:
    print("No results found")

## 8. Compare Normal Search vs Semantic Search

### What You'll See

This cell performs **both search types** on the same query and compares the results:

- **🔵 Normal Keyword Search**: Matches exact words in your documents
- **🟢 Semantic Search**: Understands meaning and context, ranks by relevance

### Which One is Better?

It depends on your use case:
- **Normal Search**: Fast, good for exact matches ("John Smith")
- **Semantic Search**: Smarter, better at understanding intent ("Who is the main character?")

**For RAG applications**: Semantic Search usually gives better results because it understands meaning, not just keywords.

### Try It!

Enter a search query below and compare the results from both search methods.

In [ ]:
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType

# Define which fields you want to display
# those fields and data just for test that's why the ain't that good 
IMPORTANT_FIELDS = [
    'metadata_storage_name',      # File name
    'content',                     # Main content
    'keyphrases',                  # Key phrases
    'people',                       # People mentioned
]

user_query = input("What do you want to search for? = ")

print(f"\n{'='*70}")
print(f"QUERY: {user_query}")
print(f"{'='*70}\n")

# ============================================================================
# 1. NORMAL (KEYWORD) SEARCH
# ============================================================================
print("🔵 NORMAL KEYWORD SEARCH")
print("-" * 70)

normal_results = search_client.search(search_text=user_query, top=5)
normal_list = list(normal_results)

print(f"Results found: {len(normal_list)}\n")

if normal_list:
    for i, result in enumerate(normal_list, 1):
        print(f"  Result {i}:")
        for key, value in result.items():
            # Only show important fields
            if key in IMPORTANT_FIELDS:
                # Truncate long values for readability
                if isinstance(value, str):
                    value_str = value[:150] + "..." if len(value) > 150 else value
                else:
                    value_str = str(value)[:150] + "..." if len(str(value)) > 150 else str(value)
                print(f"    {key}: {value_str}")
        print()
else:
    print("  No results found.\n")

# ============================================================================
# 2. SEMANTIC SEARCH
# ============================================================================
print("\n" + "="*70)
print("🟢 SEMANTIC SEARCH (AI-Powered Ranking)")
print("-" * 70)

semantic_results = search_client.search(
    search_text=user_query,
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="default",
    top=5
)

semantic_list = list(semantic_results)

print(f"Results found: {len(semantic_list)}\n")

if semantic_list:
    for i, result in enumerate(semantic_list, 1):
        print(f"  Result {i}:")
        for key, value in result.items():
            # Only show important fields
            if key in IMPORTANT_FIELDS:
                # Truncate long values for readability
                if isinstance(value, str):
                    value_str = value[:150] + "..." if len(value) > 150 else value
                else:
                    value_str = str(value)[:150] + "..." if len(str(value)) > 150 else str(value)
                print(f"    {key}: {value_str}")
        print()
else:
    print("  No results found.\n")

# ============================================================================
# 3. COMPARISON
# ============================================================================
print("\n" + "="*70)
print("📊 COMPARISON")
print("-" * 70)
print(f"Normal Search Results:    {len(normal_list)} documents")
print(f"Semantic Search Results:  {len(semantic_list)} documents")
print("\n💡 Note:")
print("  - Normal Search: Matches keywords in documents")
print("  - Semantic Search: Understands meaning and context better")
print("="*70 + "\n")